In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math

from lifelines import KaplanMeierFitter
from lifelines.fitters.coxph_fitter import CoxPHFitter
from sksurv.util import Surv
from lifelines.utils import concordance_index
from sksurv.metrics import integrated_brier_score
from sksurv.metrics import cumulative_dynamic_auc
from lifelines.statistics import proportional_hazard_test
from lifelines import WeibullAFTFitter, ExponentialFitter, LogNormalAFTFitter, LogLogisticAFTFitter

from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, ComponentwiseGradientBoostingSurvivalAnalysis
from sklearn.inspection import permutation_importance

# from bart_survival import surv_bart as sb

from sklearn import set_config
set_config(display="text")

print(pa.__version__)

import sys
print(sys.version)

# for jupyter notebook, this is necessary to show plots
%matplotlib inline

In [ ]:
#specify the input directory for saving parquet files
def read_file(file):
    input_dir = '..'
    path = f'{input_dir}/{file}.parquet'
    file = pd.read_parquet(path)
    return file

# read files
vitals = read_file('vitals')
surgery = read_file('surgery')
reanimatie = read_file('reanimatie')
lab = read_file('lab')
ic = read_file('ic_opnames')
demo = read_file('demographics')

In [ ]:
reanimatie["care_order1"] = reanimatie["care_order"].astype("category")
print(reanimatie["care_order1"].cat.categories)
reanimatie["care_order1"] = reanimatie["care_order1"].cat.codes.astype('int8')

In [ ]:
# drop extra boolean columns. this is due to values are too skewed, and takes too much memory
# but should keep prefix1 pattern
def drop_bool_cols(df, prefixes):
    cols_to_drop = [
        col for col in df.columns
        if any(col.startswith(prefix) and not col[len(prefix):].isdigit() for prefix in prefixes)
    ]
    df = df.drop(columns=cols_to_drop)
    return df

In [ ]:
# this is too skewed with 'ic', and doesn't say much others
ic = drop_bool_cols(ic, ['ic_specialisme_code', 'afdelings_code'])

# this is also not representative, Klinische opname 
demo = drop_bool_cols(demo, ['opname_type_oms'])

# 2nd useful. but memory issue. will keep the hoofdverrichting_code1
surgery = drop_bool_cols(surgery, ['hoofdverrichting_code', 'prioriteit_code'])

# useful. same
demo = drop_bool_cols(demo, ['specialisme_code'])

reanimatie = drop_bool_cols(reanimatie, ['care_order'])

In [ ]:
# rename categorical vars
demo = demo.rename(columns={'specialisme_code1': 'specialisme_code'})
surgery = surgery.rename(columns={'hoofdverrichting_code1': 'hoofdverrichting_code', 'prioriteit1': 'prioriteit_code'})
surgery = surgery.drop(columns=["prioriteit"])
reanimatie = reanimatie.rename(columns={'care_order1': 'care_order'})

In [ ]:
demo["geslacht"] = demo["geslacht"].astype("category")
print(demo["geslacht"].cat.categories)
demo["geslacht"] = demo["geslacht"].cat.codes.astype('int8')

In [ ]:
demo["spoed"] = demo["spoed"].astype("category")
print(demo["spoed"].cat.categories)
demo["spoed"] = demo["spoed"].cat.codes.astype('int8')

In [ ]:
# remove boolean values because they take more memory
demo = demo.drop(['geslacht_m', 'spoed_j'], axis=1)

In [ ]:
# # execute this code if there's a convergence issue with a model
# #     if death_fullcode_first:
# #        1
# #     elif death_ic_first:
# #         2
# #     elif ic_6hr_first:
# #         3
# #     elif acute_ic_first:
# #         4
# demo['first_event'] = demo['first_event'].replace({4: 1, 2: 1})
# demo['first_event'].value_counts()

In [ ]:
# demo['count_combined'] = demo['count_death_fullcode'] | demo['count_death_ic'] | demo['count_acute_ic']
# demo = demo.drop(['count_death_fullcode', 'count_death_ic', 'count_acute_ic'], axis=1)

In [ ]:
# # maybe use count instead
# # necessary to avoid convergence issue
# # ic_6hr
# demo['is_combined'] = demo['death_fullcode'] | demo['death_ic'] | demo['acute_ic']
# demo = demo.drop(['death_fullcode', 'death_ic', 'acute_ic'], axis=1)
# demo['is_combined'] = demo['is_combined'].fillna(False)

In [ ]:
# demo['is_combined'].isna().sum()

# Selecting Subset of Population

In [ ]:
# why there are even duplicated rows in demo? no clue
demo[demo.duplicated(keep=False)]
demo = demo.drop_duplicates()
ic = ic.drop_duplicates()
surgery = surgery.drop_duplicates()

In [ ]:
demo['is_first'].value_counts()

In [ ]:
group_keys = demo.groupby('first_event').groups.keys()
group_keys

In [ ]:
demo.groupby('first_event').size()

In [ ]:
demo['first_event'].value_counts(normalize=True) * 100

In [ ]:
# # stratified sampling: proportionate sampling with fixed seed
# # if under 0.01, there are way less samples for some cases
# demo = demo.groupby('first_event', group_keys=True).apply(lambda x: x.sample(frac=0.01, random_state=42))

In [ ]:
# frac 0.001 to Cox.
# 0.01 (?) works with merging data but doesn't fit in Cox
# just in case to have unique opname id per group, which will be expanded while merging
def get_sample(group, frac=0.001, min_num=3, random_state=1):
    group_unique = group.drop_duplicates(subset="opname_id")
    
    # get sample size
    n = math.ceil(len(group_unique) * frac)
    
    # ensures minimum numbers
    n = max(n, min_num)
    
    # ensures not exceeding max number
    n = min(n, len(group_unique))
    
    return group_unique.sample(n=n, random_state=random_state)

demo = demo.groupby('first_event', group_keys=True).apply(get_sample)

In [ ]:
demo[demo['first_event']==4]

In [ ]:
demo['first_event'].value_counts(normalize=True) * 100

In [ ]:
demo['first_event'].value_counts()

In [ ]:
demo['opname_id'].nunique()

In [ ]:
demo.iloc[0]['p_id']

In [ ]:
demo['is_first'].value_counts()

In [ ]:
demo.info()

In [ ]:
columns_drop = [
    'death_fullcode', 'death_ic', 'acute_ic', 'ic_6hr', 
    'care_order_full_code', 
    'ic_ontslag_datum_tijd', 'ic_opname_datum_tijd', 
    'ok_eind_datum_tijd', 'prioriteit_code_acute', 'death_fullcode_first', 
    'death_ic_first', 'ic_6hr_first', 'acute_ic_first', 'overlijdens_datum', 
    #'count_death_fullcode', 'count_death_ic', 'count_ic_6hr', 'count_acute_ic', 
    #'first_event',
    #'opname_datum_tijd', 'ontslag_datum_tijd'
]

demo = demo.drop(columns=columns_drop)

In [ ]:
# creating a datetime variable from separate integer variables due to bug of parquet
def create_datetime(df, original_col):
    df = df.drop(original_col, axis=1)
    
    year_col = f"{original_col}_year"
    month_col = f"{original_col}_month"
    day_col = f"{original_col}_day_of_month"
    hour_col = f"{original_col}_hour"
    
    col_map = {
        year_col: 'year',
        month_col: 'month',
        day_col: 'day',
        hour_col: 'hour'
    }
    
    df = df[list(col_map.keys())].rename(columns=col_map)
    return pd.to_datetime(df)

In [ ]:
demo['opname_datum_tijd'] = create_datetime(demo, 'opname_datum_tijd')
demo['ontslag_datum_tijd'] = create_datetime(demo, 'ontslag_datum_tijd')

surgery['ok_begin_datum_tijd'] = create_datetime(surgery, 'ok_begin_datum_tijd')
surgery['ok_eind_datum_tijd'] = create_datetime(surgery, 'ok_eind_datum_tijd')

reanimatie['vanaf_datum'] = create_datetime(reanimatie, 'vanaf_datum')

lab['lab_datum_tijd'] = create_datetime(lab, 'lab_datum_tijd')

ic['ic_opname_datum_tijd'] = create_datetime(ic, 'ic_opname_datum_tijd')
ic['ic_ontslag_datum_tijd'] = create_datetime(ic, 'ic_ontslag_datum_tijd')

vitals['meting_datum_tijd'] = create_datetime(vitals, 'meting_datum_tijd')

In [ ]:
# filling where event == 0 as stay time and convert all in hour unit
time_diff = demo['ontslag_datum_tijd'] - demo['opname_datum_tijd']
time_diff = (time_diff.dt.total_seconds() / 60).round().astype('UInt32')
demo.loc[demo['first_event'] == 0, 'time_to_first_event'] = time_diff
demo['time_to_first_event'] = (demo['time_to_first_event'] / 60).round().astype('UInt16')

# I think we don't do anything with this period, as it is indicated in time_to_first_event
demo = demo.drop(['opname_datum_tijd', 'ontslag_datum_tijd'], axis=1)

In [ ]:
vitals.columns

In [ ]:
# same reason
dataframes = [demo, surgery, reanimatie, ic, lab]

def drop_time_cols(dataframes):
    suffixes = ['_year', '_month', '_day_of_month', '_hour']
    for df in dataframes:
        cols_drop = [col for col in df.columns if any(col.endswith(suffix) for suffix in suffixes)]
        df.drop(columns=cols_drop, inplace=True)
    
drop_time_cols(dataframes)

In [ ]:
vitals = vitals.drop(['meting_datum_tijd_year',
       'meting_datum_tijd_month', 'meting_datum_tijd_day_of_month',
       'meting_datum_tijd_hour'], axis=1)

In [ ]:
# Not necessary
# def get_duration(df, start_col, end_col, new_col):
#     df = df[df[start_col] <= df[end_col]].copy()
#     time_diff = (df[end_col] - df[start_col])
#     df[new_col] = (time_diff.dt.total_seconds() / 3600).round()
#     df[new_col] = df[new_col].astype('int8')
#     df.drop([start_col, end_col], axis=1, inplace=True)
#     return df

In [ ]:
# surgery = get_duration(surgery, 'ok_begin_datum_tijd', 'ok_eind_datum_tijd', 'surgery_duration')

In [ ]:
# surgery['surgery_duration'].max()

In [ ]:
demo['is_first'] = demo['is_first'].astype('int8')

In [ ]:
demo['first_event'] = demo['first_event'].astype('int8')

In [ ]:
# demo = demo.drop(['first_event'], axis=1)

In [ ]:
# huh, it cannot handle 'UInt' or 'Int' type
demo['time_to_first_event'] = demo['time_to_first_event'].astype('int32')

# demo = demo.set_index(['p_id', 'opname_id'])

# Cleaning some rows

In [ ]:
def missing_p(df):
    p = df.isna().mean() * 100
    return p

In [ ]:
missing_p(lab)

In [ ]:
missing_p(vitals)

In [ ]:
# if we adjust the threshold here, then i'll remove some boolean variables as a compensation for memory
# and what if we train the datetime columns?
def drop_high_missing_cols(df, t=50):
    p = missing_p(df)
    columns_to_drop = p[p > t].index
    df_dropped = df.drop(columns=columns_to_drop)
    return df_dropped

In [ ]:
vitals = drop_high_missing_cols(vitals)
vitals.info()

In [ ]:
lab = drop_high_missing_cols(lab)
lab.info()

In [ ]:
vitals = vitals.drop(['meting_datum_tijd'], axis=1)
vitals.info()

In [ ]:
surgery.info()

In [ ]:
surgery = surgery.drop(['operatie_id', 'ok_begin_datum_tijd', 'ok_eind_datum_tijd'], axis=1)

In [ ]:
reanimatie.info()

In [ ]:
reanimatie = reanimatie.drop(['vanaf_datum'], axis=1)

In [ ]:
ic.info()

In [ ]:
ic['ic_6hr'] = ic['ic_6hr'].astype('boolean')

In [ ]:
# better to use the count. it's more admission specific
ic = ic.drop(['ic_6hr'], axis=1)

In [ ]:
ic = ic.drop(['ic_opname_datum_tijd', 'ic_ontslag_datum_tijd'], axis=1)

In [ ]:
lab = lab.drop(['lab_datum_tijd'], axis=1)

In [ ]:
# replacing with nominal values for other variables

# ANIONGAP	Aniongap	8
# BICARB	Bicarbonate	27
# BPDIA	Diastolic blood pressure	70
# BPSYS	Systolic blood pressure	110
# GLUCOSE	Glucose	5.5
# HEMAT	Hematocrit	0.34
# HRTRT	Heart rate	80
# LACT	Lactate	0
# BUN	Blood urea nitrogen	3.5
# CREAT	Creatinine	115
# AGE	Age	17
# SAT	Saturation	99
# RSPRT	Respiration rate	12
# SODIUM	Sodium	140
# TEMP (C)	Temprature	37
# TROP	Troponin	0
# WBC	White blood cell count	5
# PH	Blood Ph	7.41
# BIL	Bilirubin	8.2
# ALB	Albumin	34
# PAC	PaCO2	39
# PAO	PaO2	82

nominal_values = {
    'kreatinine': 115,
    'natrium': 140,
    'ureum': 3.5,
    'hr_meet_waarde1': 80,
    'nibp_meet_waarde1': 110,
    'nibp_meet_waarde2': 70,
    'nibp_meet_waarde3': 83.33,
    'resp_meet_waarde1': 12,
    'spo2_meet_waarde1': 99,
    'temp_meet_waarde1': 37
}

vitals = vitals.fillna(value=nominal_values)
lab = lab.fillna(value=nominal_values)

In [ ]:
# convert dtypes (UInt, Int to int as we're removing any NaN values and they take way less memory usage)

dataframes = [vitals, surgery, reanimatie, lab, ic, demo]

dtype_mapping = {
    'Int8': 'int8',
    'Int16': 'int16',
    'Int32': 'int32',
    'Int64': 'int64',

    'UInt8': 'int8',
    'UInt16': 'int16',
    'UInt2': 'int32',
    'UInt64': 'int64',
}

for df in dataframes:
    for col in df.columns:
        current_dtype = str(df[col].dtype)
        if current_dtype in dtype_mapping:
            df[col] = df[col].astype(dtype_mapping[current_dtype])
    print("===========")
    print(df.dtypes)

# Merging other datasets

In [ ]:
def merge_dataframes(df1, df2):
    merged_df = pd.merge(df1, df2, on='opname_id', how='left', suffixes=('_x', '_y'))
    merged_df = merged_df.drop(columns=[col for col in merged_df.columns if col.endswith('_y')])
    merged_df = merged_df.rename(columns=lambda x: x.replace('_x', ''))
    
    # if no values, then fill in -1
    for col in ["hoofdverrichting_code", "prioriteit_code", "care_order", "m_year", "m_month", "m_day", "m_hour"]:
        if col in merged_df.columns:
            if col == "m_year":
                merged_df[col] = merged_df[col].fillna(-1).astype("int16")
            else:
                merged_df[col] = merged_df[col].fillna(-1).astype("int8")
    return merged_df

In [ ]:
merged_df = merge_dataframes(demo, surgery)
del demo, surgery

In [ ]:
# nothing's gonna be involved from here
# merged_df = merge_dataframes(merged_df, ic)
# del ic

In [ ]:
merged_df = merge_dataframes(merged_df, reanimatie)
del reanimatie

In [ ]:
merged_df = merge_dataframes(merged_df, lab)
del lab

In [ ]:
merged_df = merge_dataframes(merged_df, vitals)
del vitals

In [ ]:
# not used as i removed all bool/boolean values
def replace_false(df):
    for column in df.columns:
        if pd.api.types.is_bool_dtype(df[column]):
            df[column] = df[column].fillna(False)
    return df

In [ ]:
df = replace_false(merged_df)

In [ ]:
# bool_columns = [
#     "prioriteit_code_acute",
#     "prioriteit_code_elective",
#     "prioriteit_code_unknown",
#     "care_order_dnr",
#     "care_order_full_code",
#     "care_order_partial"
# ]
# df[bool_columns] = df[bool_columns].astype(bool)  
# df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
# replacing with nominal values for other variables

# ANIONGAP	Aniongap	8
# BICARB	Bicarbonate	27
# BPDIA	Diastolic blood pressure	70
# BPSYS	Systolic blood pressure	110
# GLUCOSE	Glucose	5.5
# HEMAT	Hematocrit	0.34
# HRTRT	Heart rate	80
# LACT	Lactate	0
# BUN	Blood urea nitrogen	3.5
# CREAT	Creatinine	115
# AGE	Age	17
# SAT	Saturation	99
# RSPRT	Respiration rate	12
# SODIUM	Sodium	140
# TEMP (C)	Temprature	37
# TROP	Troponin	0
# WBC	White blood cell count	5
# PH	Blood Ph	7.41
# BIL	Bilirubin	8.2
# ALB	Albumin	34
# PAC	PaCO2	39
# PAO	PaO2	82

nominal_values = {
    'kreatinine': 115,
    'natrium': 140,
    'ureum': 3.5
}

df = df.fillna(value=nominal_values)

In [ ]:
df.isna().sum()

In [ ]:
# remove where the measurement records are non existing
df = df.dropna(subset=['nibp_meet_waarde1'])
df = df[df['m_year'] != -1]
df.isna().sum()

In [ ]:
df.info()

In [ ]:
pd.set_option('display.max_seq_items', None)
df.columns

In [ ]:
df.info(verbose=True, max_cols=None, show_counts=True)

In [ ]:
dataframes = [df]

# initialize titles
titles = ["df_for_sa_rand1"]

# specify the output directory for saving parquet files
output_dir = '..'

# create the folder if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# save each dataframe as a parquet file
for df, title in zip(dataframes, titles):
    output_path = os.path.join(output_dir, f"{title}.parquet")

    # check if the file already exists
    if not os.path.isfile(output_path):
        try:
            # save dataframe to parquet file
            df.to_parquet(output_path)
            
        except Exception as e:
            print(f"Failed to save {title}: {e}.")
            continue

In [ ]:
def read_file(file):
    input_dir = '..'
    path = f'{input_dir}/{file}.parquet'
    file = pd.read_parquet(path)
    return file

# read files
df1 = read_file('df_for_sa_rand1')
df1.info()

# Cleaning Done HERE

The below cells are for running survival analysis method.    
And cox was to check convergence issue. 

# THE BELOW CODE IS JUST TO CHECK CONVERGENCE ISSUE

# Train Test

In [ ]:
# # should've changed the 'is_first' as int..already did above

# extra_cols = ['opname_id', 'p_id']

# df1 = df.drop(columns=extra_cols)

In [ ]:
# should've changed the 'is_first' as int..already did above

df = df.set_index(['p_id', 'opname_id'])
df = df.reset_index(drop=True)
df1 = df.copy()

In [ ]:
# df1['care_order_partial'].value_counts()
# df1['prioriteit_code_unknown'].value_counts()

In [ ]:
# df1 = df1.drop(['prioriteit_code_unknown', 'care_order_partial', 'first_event'], axis=1)

In [ ]:
train, test = train_test_split(df1, test_size=0.2, random_state=42)

duration_col = 'time_to_first_event'
event_col = 'is_first'

X = df1.drop([event_col, duration_col], axis=1)
y = df1[[event_col, duration_col]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Cox

In [ ]:
train.isna().sum()

In [ ]:
# fit cox and get the estimation of hourly survival probability
cph = CoxPHFitter(alpha=0.05, penalizer=0.01)
cph.fit(train, duration_col='time_to_first_event', event_col='is_first', batch_mode=True)

# define it to the max range for all entries
times = list(range(0, max(df1['time_to_first_event']) + 1))

survival = cph.predict_survival_function(X_test, times=times)
survival

# Evaluation

In [ ]:
# # If your objective is prediction, you can focus on the loss metric
# cph.check_assumptions(df1, p_value_threshold=0.05)

In [ ]:
df1['is_first'].value_counts()

In [ ]:
# # NOT NECESSARY BECAUSE THIS ONE WILL HAVE 'AVG_SURV PROBA' Column

# dataframes = [df1]

# # initialize titles
# titles = ["cox"]

# # specify the output directory for saving parquet files
# output_dir = '/..'

# # create the folder if it does not exist
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # save each dataframe as a parquet file
# for df, title in zip(dataframes, titles):
#     output_path = os.path.join(output_dir, f"{title}.parquet")

#     # check if the file already exists
#     if not os.path.isfile(output_path):
#         try:
#             # save dataframe to parquet file
#             df.to_parquet(output_path)
            
#         except Exception as e:
#             print(f"Failed to save {title}: {e}.")
#             continue